In [ ]:
from shapely.geometry import LineString
from shapely.ops import transform
from pyproj import Transformer
import utils

get_alt = utils.get_get_alt('./data/_hakone.vrt')
to_meter = Transformer.from_crs("EPSG:4612", "EPSG:2451", always_xy=True).transform
to_latlon = Transformer.from_crs("EPSG:2451", "EPSG:4612", always_xy=True).transform


def get_cartgraphics(geom, height, speed):
    def get_distances(geom):
        lengths = [
            LineString(geom.coords[i : i + 2]).length
            for i in range(len(geom.coords) - 1)
        ]
        return [0] + [sum(lengths[: i + 1]) for i in range(len(lengths))]

    dists = get_distances(transform(to_meter, geom))
    speed_mps = speed * 1000 / 3600
    total_secs = dists[-1] / speed_mps
    secs = [total_secs * (d / dists[-1]) for d in dists]
    sec_coords = [
        (sec, x, y, get_alt(x, y) + height) for sec, (x, y) in zip(secs, geom.coords)
    ]
    return [a for sec_coord in sec_coords for a in sec_coord]

In [ ]:
def get_header(name):
    return {"id": "document", "name": name, "version": "1.0"}


def get_body(aid, name, epoch, flat_poss_list, model):
    return {
        "id": aid,
        "name": name,
        "position": {
            "epoch": epoch.strftime("%Y-%m-%dT%H:%M:%S+09"),
            "cartographicDegrees": flat_poss_list,
        },
        "orientation": {"velocityReference": "%s#position" % aid},
        "forwardExtrapolationType": "None",
        "backwardExtrapolationType": "None",
        "model": model,
    }

In [ ]:
import json
from datetime import datetime, timedelta
import random as rd
import geopandas as gpd
from pyproj import Transformer
from shapely.ops import substring, transform

GEOID = 36.7071
HEIGHT = GEOID + 2.2

to_meter = Transformer.from_crs("EPSG:4612", "EPSG:2451", always_xy=True).transform
to_latlon = Transformer.from_crs("EPSG:2451", "EPSG:4612", always_xy=True).transform

# make yumoto down czml
# box = {
#     "dimensions": {"cartesian": [4.8, 2, 2]},
#     "material": {"solidColor": {"color": {"rgba": [216, 216, 216, 0xFF]}}},
#     "shadows": "ENABLED",
# }
models = [
    {"gltf": "./vehicle.glb"},
    {"gltf": "./vehicle.glb"},
    {"gltf": "./vehicle.glb"},
    {"gltf": "./vehicle.glb"},
    {"gltf": "./vehicle.glb"},
    {"gltf": "./vehicle.glb"},
    {"gltf": "./vehicle.glb"},
    {"gltf": "./truck.glb"},
    {"gltf": "./truck.glb"},
    {"gltf": "./bus.glb"},
]
seeds = [
    [6, 180],
    [7, 120],
    [8, 90],
    [9, 120],
    [10, 120],
    [11, 120],
]


def get_secs(hour, interval):
    intervals = [rd.randint(3, interval) for _ in range(300)]
    return [
        a + hour * 3600
        for i, _ in enumerate(intervals)
        if (a := sum(intervals[:i])) < 3600
    ]


day = datetime.fromisoformat("2022-11-19")
secs = sum([get_secs(*seed) for seed in seeds], [])
epochs = [day + timedelta(seconds=sec) for sec in secs]

geom = gpd.read_file("./data/_rdcl_1_yumoto.geojson").geometry[0]
for_geom = transform(to_latlon, transform(to_meter, geom).parallel_offset(-2))
for_carts = get_cartgraphics(for_geom, HEIGHT, 30)
back_geom = substring(
    transform(to_latlon, transform(to_meter, geom).parallel_offset(2)),
    1,
    0,
    normalized=True,
)
back_carts = get_cartgraphics(back_geom, HEIGHT, 30)

packets = (
    [get_header("一般車両")]
    + [
        get_body(f"{i*2+1}", "上り", epoch, for_carts, model=models[i % 10])
        for i, epoch in enumerate(epochs)
    ]
    + [
        get_body(f"{i*2+2}", "下り", epoch, back_carts, model=models[i % 10])
        for i, epoch in enumerate(epochs)
    ]
)
open("./dst/vehicle_yumoto_czml.json", "w").write(
    json.dumps(packets, indent=2, ensure_ascii=False)
)